In [1]:
from vm.mnemonic import WProtectMnemonic
from vm.analysis import PEAnalysis, WProtectEmulator, WProtectTracer
import vm.obfuscation
from vm.rule_generator import generate_rules
from semantic.flow import WProtectControlFlow, UnknownInstructionError, PositionInstructionError

In [2]:
center2 = WProtectEmulator("if_nz/if.wp.exe")
center2.find()
center = center2.recover_mnemonics(center2.offset, WProtectMnemonic)[1]
WProtectMnemonic.consolidate_names(center)

[INFO]	PE loaded
[INFO]	reachable code analysis done


[INFO]	WProtect vm mnemonics found at offset 0x40a96e


[INFO]	Mnemonics loaded


In [3]:
def printer(offset, instruction, args, off):
    t.add_node(offset, instruction, args, off)
    print "%s\t%s\t%s" % (hex(offset), map(hex, off),instruction.name)

t = WProtectControlFlow(center)
instance = WProtectTracer(center2, center)

machine = [instance]
index = 0
while index < len(machine):
    try:
        tmp = machine[index].step(printer)
        machine += tmp
    except PositionInstructionError as err:
        print err
    index += 1

import networkx as nx
import matplotlib.pyplot as plt
    
nx.draw_spring(t.get_simple_control_flow(), node_size=12)
plt.show()

d_push_reg ['0x14']
0x40b352L	['0x40b350L']	d_push_reg
d_push_reg ['0x3c']
0x40b350L	['0x40b34eL']	d_push_reg
d_push_reg ['0x30']
0x40b34eL	['0x40b34cL']	d_push_reg
d_push_reg ['0xc']
0x40b34cL	['0x40b34aL']	d_push_reg
d_push_reg ['0x0']
0x40b34aL	['0x40b348L']	d_push_reg
d_push_reg ['0x2c']
0x40b348L	['0x40b346L']	d_push_reg
d_push_reg ['0x34']
0x40b346L	['0x40b344L']	d_push_reg
d_push_reg ['0x18']
0x40b344L	['0x40b342L']	d_push_reg
d_push_reg ['0x20']
0x40b342L	['0x40b340L']	d_push_reg
d_push_imm ['0x123452fd']
0x40b340L	['0x40b33bL']	d_push_imm
d_pop_reg ['0x31']
0x40b33bL	['0x40b339L']	d_pop_reg
b_push_imm_sx ['0xfa']
0x40b339L	['0x40b337L']	b_push_imm_sx
d_add []
0x40b337L	['0x40b336L']	d_add
d_push_reg ['0x24']
0x40b336L	['0x40b334L']	d_push_reg
d_read_mem []
0x40b334L	['0x40b333L']	d_read_mem
push_stack_top_base []
0x40b333L	['0x40b332L']	push_stack_top_base
d_read_mem []
0x40b332L	['0x40b331L']	d_read_mem
d_nand []
0x40b331L	['0x40b330L']	d_nand
d_push_reg ['0x5c']
0x40b330L	['

ValueError: need more than 0 values to unpack

In [ ]:
out = {}
compiled = t.compile_blocks(out)

In [ ]:
compiled.keys()

In [ ]:
from keystone import Ks, KS_ARCH_X86, KS_MODE_32

In [ ]:
code = b"start:;INC ecx; DEC edx; JMP start"
ks = Ks(KS_ARCH_X86, KS_MODE_32)
encoding, count = ks.asm(code)

In [ ]:
map(hex, encoding)

In [ ]:
import semantic.translator
reload(semantic.translator)
asm = semantic.translator.W_READ_STACK % 10
print
print map(lambda x: "%02x" % x, ks.asm(asm)[0])

In [ ]:
semantic.translator.B_READ_STACK % 10